In [2]:
#open pandas DF 
import pandas as pd
data= pd.read_csv("../Raw_Data/NCRMP_USVI_raw.csv")
print(data.columns)

Index(['time', 'latitude', 'longitude', 'REGION', 'REGION_DESCRIPTION',
       'PRIMARY_SAMPLE_UNIT', 'STATION_NR', 'YEAR', 'MONTH', 'DAY', 'Date_UTC',
       'HABITAT_CD', 'HABITAT_TYPE', 'STRAT', 'STRAT_Description',
       'RUGOSITY_CD', 'WTD_RUG', 'MEAN_RUG', 'MAPGRID_NR', 'SUB_REGION_NAME',
       'SUB_REGION_NAME_DESCRIPTION', 'SUB_REGION_NR', 'ZONE_NAME', 'ZONE_NR',
       'MPA_NAME', 'MPA_NR', 'ADMIN', 'Administration_Description', 'PROT',
       'DEPTH_STRAT', 'DEPTH_STRAT_DESCRIPTION', 'MIN_DEPTH', 'MAX_DEPTH',
       'METERS_COMPLETED', 'SPECIES_CD', 'SPECIES_NAME', 'N', 'JUV',
       'MAX_DIAMETER', 'PERP_DIAMETER', 'HEIGHT', 'OLD_MORT', 'RECENT_MORT',
       'BLEACH_CONDITION', 'DISEASE', 'accession_url'],
      dtype='object')


/var/folders/17/9kvd528n47j282gcln4v5zvc0000gp/T/ipykernel_2071/3413153539.py:3: DtypeWarning: Columns (1,2,17,31,32,33,38,39,40,41,42,45) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv("../Raw_Data/NCRMP_USVI_raw.csv")


In [3]:
# isolating Genus for counting
data = data[["SPECIES_CD","SPECIES_NAME","DISEASE"]]
data = data.dropna(how='all').reset_index(drop=True)
data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,AGA GRAH,Agaricia grahamae,NaN
1,AGA HUMI,Agaricia humilis,NaN
2,AGA LAMA,Agaricia lamarcki,NaN
3,AGA SPE.,Agaricia spp,NaN
4,COL NATA,Colpophyllia natans,NaN
...,...,...,...
175500,SID SIDE,Siderastrea siderea,A
175501,SID SIDE,Siderastrea siderea,A
175502,SID SPE.,Siderastrea spp,NaN
175503,SOL BOUR,Solenastrea bournoni,NaN


In [4]:
# select Disease column and setting empty cells to healthy 
data['DISEASE'][data['DISEASE'].isnull()] = "Unknown"
data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,AGA GRAH,Agaricia grahamae,Unknown
1,AGA HUMI,Agaricia humilis,Unknown
2,AGA LAMA,Agaricia lamarcki,Unknown
3,AGA SPE.,Agaricia spp,Unknown
4,COL NATA,Colpophyllia natans,Unknown
...,...,...,...
175500,SID SIDE,Siderastrea siderea,A
175501,SID SIDE,Siderastrea siderea,A
175502,SID SPE.,Siderastrea spp,Unknown
175503,SOL BOUR,Solenastrea bournoni,Unknown


In [5]:
#Count of corals by genus and Disease column 
Grouped_data = data.groupby(['SPECIES_NAME','DISEASE']).agg(Disease_count = ("DISEASE", "count")).reset_index()

Grouped_data

,SPECIES_NAME,DISEASE,Disease_count
0,Acropora cervicornis,A,70
1,Acropora cervicornis,P,3
2,Acropora cervicornis,S,2
3,Acropora cervicornis,Unknown,2089
4,Acropora palmata,A,106
...,...,...,...
193,Stephanocoenia intersepta,F,1
194,Stephanocoenia intersepta,P,16
195,Stephanocoenia intersepta,Unknown,1462
196,Tubastraea coccinea,A,43


In [6]:
from numpy import isfinite
from math import isnan
disease_df = data
unique_diseases = set(disease_df["DISEASE"].unique())

df = data
unique_species = set(df["SPECIES_NAME"].unique())
unique_species = list(unique_species)
print(unique_species)
print(unique_diseases)

disease_table = pd.DataFrame(0, index=unique_species,columns=unique_diseases)
for species_name in unique_species:
    for disease in unique_diseases: 
        disease_subtable = df.loc[(df['SPECIES_NAME'] == species_name) & (df['DISEASE'] == disease)]
        empty = bool(disease_subtable.empty)
        if not empty:
            count = disease_subtable['DISEASE'].count()
        
            disease_table.loc[species_name, disease] = count
disease_table.sort_index()

disease_table = disease_table.rename(columns ={'A':'healthy_count', 'P':'total_diseased_count', 'Unknown':'unknown_count'})
disease_table

disease_table["total_diseased_percent"] = disease_table["total_diseased_count"]/(disease_table["healthy_count"]+ disease_table["total_diseased_count"])

disease_table = disease_table.drop(columns=['F','S'])
disease_table = disease_table.rename_axis('species')
disease_table

['Agaricia fragilis', 'Isophyllastrea rigida', 'Manicina areolata', 'Porites divaricata', 'Diploria labyrinthiformis', 'Porites spp', 'Oculina diffusa', 'Scolymia cubensis', 'Meandrina spp', 'Acropora cervicornis', 'Mussa angulosa', 'Siderastrea radians', 'Mycetophyllia lamarckiana', 'Solenastrea spp', 'Agaricia lamarcki', 'Orbicella annularis species complex', 'Meandrina danae', 'Orbicella spp', 'Montastraea cavernosa', 'Agaricia grahamae', 'Madracis spp', 'Colpophyllia natans', 'Isophyllia sinuosa', 'Solenastrea bournoni', 'Helioceris cucullata', 'Mycetophyllia aliciae', 'Porites branneri', 'Stephanocoenia intersepta', 'Scleractinia spp', 'Eusmilia fastigiata', 'Favia fragum', 'Orbicella annularis', 'Siderastrea siderea', 'Cladocora arbuscula', 'Agaricia tenuifolia', 'Madracis decactis', 'Acropora palmata', 'Meandrina jacksoni', 'Agaricia humilis', 'Agaricia spp', 'Siderastrea spp', 'Acropora prolifera', 'Dichocoenia stokesii', 'Pseudodiploria clivosa', 'Madracis pharensis', 'Other c

,unknown_count,total_diseased_count,healthy_count,total_diseased_percent
species,,,,
Agaricia fragilis,2160,0,150,0.000000
Isophyllastrea rigida,2058,0,34,0.000000
Manicina areolata,2206,0,51,0.000000
Porites divaricata,2029,5,612,0.008104
Diploria labyrinthiformis,2067,3,303,0.009804
...,...,...,...,...
Agaricia agaricites,1871,30,8423,0.003549
Mycetophyllia spp,1330,0,8,0.000000
Orbicella faveolata,2016,73,3115,0.022898


In [7]:
disease_table.to_csv("../Processed_Data/NCRMP_USVI_corals.csv",index_label="species_name")